# 전자서명

## ✅ 전자서명 vs 일반 암호화 비교

| 항목 | 일반 암호화 | 전자서명 |
| --- | --- | --- |
| 암호화에 사용되는 키 | **수신자의 공개키** (PU_B) | **송신자의 개인키** (PR_A) |
| 복호화에 사용되는 키 | 수신자의 개인키 (PR_B) | **송신자의 공개키** (PU_A) |
| 목적 | 비밀 유지 (기밀성) | 위조 방지, 발신자 인증 (무결성 + 인증) |

---

## 🧠 핵심 개념 요약

- 🔓 **공개키**는 **공유용** (누구나 알 수 있음)
- 🔐 **개인키**는 **본인만 보관**

---

## 🔐 전자서명 흐름

1. **송신자 A**가 메시지에 대해 해시함 (예: SHA-256)
2. 그 해시값을 **자신의 개인키(PR_A)로 암호화** → 전자서명 생성
3. 수신자는 A의 **공개키(PU_A)**로 복호화하여 해시값 확인
4. 받은 메시지에서 직접 해시 계산 → 비교

→ 해시값이 같다면 **발신자 인증 + 무결성 확인 성공**

---

## 💻 전자서명 파이썬 예시 (RSA + SHA-256)

In [2]:
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA

# 송신자 A의 키 쌍
key = RSA.generate(2048)
private_key_A = key
public_key_A = key.publickey()

# 메시지와 해시 생성
message = b"hello!"
hash = SHA256.new(message)

# 송신자 A가 개인키로 서명
signature = pkcs1_15.new(private_key_A).sign(hash)

# 수신자가 공개키로 서명 검증
try:
    pkcs1_15.new(public_key_A).verify(hash, signature)
    print("서명 검증 성공 ✅")
except (ValueError, TypeError):
    print("서명 검증 실패 ❌")


서명 검증 성공 ✅


## 🧾 시험 요점 정리

- “**전자서명은 개인키로 암호화하고, 공개키로 복호화한다**” → ✅ 맞는 말
- **기밀성**이 목적일 땐 → 공개키 암호화 / 개인키 복호화
- **무결성과 인증**이 목적일 땐 → **개인키 서명 / 공개키 검증**

## 🔐 DSS(Digital Signature Standard) 핵심 구조

- 사용 알고리즘: **DSA (Digital Signature Algorithm)**
- DSA는 ElGamal 기반이지만 **암호화는 안 하고 서명 전용**

---

## 🔑 용어 정리

| 역할 | 설명 |
| --- | --- |
| **서명자 = 송신자 (발신자)** | 메시지를 서명하고 보냄 |
| **검증자 = 수신자 (수신자)** | 서명을 검증하고 메시지의 무결성과 신원을 확인 |

---

## 🧠 서명 생성 vs 검증 구성

### ✅ 서명 생성 (서명자 / 송신자)

- 입력: 메시지 $M$
- 과정:
    1. 메시지 $M$을 해시함 → $H(M)$
    2. 랜덤 값 $k$ 선택
    3. **서명값 (r, s)** 계산:
        - $r = (g^k \mod p) \mod q$
        - $s = k^{-1}(H(M) + x \cdot r) \mod q$
            - 여기서 xxx: 서명자의 비밀키
- 결과: $(r,s)$ 서명 쌍 전송 → **메시지 + (r, s)**

✅ → **서명 생성 함수 2개: r 계산 / s 계산**

---

### ❌ 틀린 보기 설명 (검증자 / 수신자)

- 보기 문장:

    “검증자는 서명으로부터 추출한 값과 메시지로부터 얻은 해시값을 비교하여 일치하는가를 확인함으로써 서명을 검증한다.” → ❌

- ❌ 이유: DSS는 **단순 비교 방식이 아님**
    - 서명값(r, s), 공개키, 파라미터 등을 이용해 **검증용 계산식**을 돌림

### ✅ 검증 절차 (검증자 / 수신자)

- 입력: $M, r, s,$ 공개키 $y$
- 과정:
    1. $H(M)$ 계산
    2. $w = s^{-1} \mod q$
    3. $u_1 = H(M) \cdot w \mod q$, $u_2 = r \cdot w \mod q$
    4. $v = ((g^{u_1} \cdot y^{u_2}) \mod p) \mod q$
- **검증:** $v==r$ → 성공

✅ → **검증 함수는 하나지만 내부에 계산이 많음**

---

## 📌 요약 포인트 (시험 대비)

| 요소 | 설명 |
| --- | --- |
| 서명자 = 송신자 | 비밀키로 서명 (r, s 생성) |
| 검증자 = 수신자 | 공개키로 서명 검증 (v =? r) |
| 서명 = 두 요소 (r, s) | 메시지와 함께 전송 |
| 검증 = 해시와 비교 아님 | 수식 계산 후 v == r 확인 |

---

## ❓ 보기 정리 예

> 다음 중 DSS 서명 검증에 대한 설명으로 옳은 것은?
>
1. 메시지 해시값과 서명값을 단순 비교하여 서명을 검증한다. → ❌
2. 서명자는 서명 생성 시 두 개의 값을 생성하여 함께 전송한다. → ✅
3. DSS는 공개키로 메시지를 암호화한다. → ❌
4. 검증자는 서명값(r, s), 메시지, 공개키로 계산된 값과 r을 비교한다. → ✅

## **DSS(디지털 서명 표준)**을 이용한 파이썬 예시

In [4]:
# 메시지를 ASCII 문자로 수정
message = b"This is an example of DSS signature."

from Crypto.PublicKey import DSA
from Crypto.Signature import DSS
from Crypto.Hash import SHA256

# 1. 서명자 키 생성 (개인키 + 공개키)
key = DSA.generate(2048)
public_key = key.publickey()

# 2. 메시지 해시
hash_obj = SHA256.new(message)

# 3. 서명 생성 (r, s 쌍 포함됨)
signer = DSS.new(key, 'fips-186-3')
signature = signer.sign(hash_obj)

# 4. 검증자 측 - 메시지 해시 후 서명 검증
verifier = DSS.new(public_key, 'fips-186-3')
try:
    verifier.verify(hash_obj, signature)
    result = "서명 검증 성공 ✅"
except ValueError:
    result = "서명 검증 실패 ❌"

result


'서명 검증 성공 ✅'

## ✅ 실행 결과

> 서명 검증 성공 ✅
>

---

## 📄 코드 설명 요약

```python
from Crypto.PublicKey import DSA
from Crypto.Signature import DSS
from Crypto.Hash import SHA256

```

1. **키 생성**

    ```python
    key = DSA.generate(2048)
    public_key = key.publickey()

    ```

2. **메시지 해싱**

    ```python
    message = b"This is an example of DSS signature."
    hash_obj = SHA256.new(message)
    ```

3. **서명 생성 (송신자, 서명자)**

    ```python
    signer = DSS.new(key, 'fips-186-3')
    signature = signer.sign(hash_obj)

    ```

4. **서명 검증 (수신자, 검증자)**

    ```python
    verifier = DSS.new(public_key, 'fips-186-3')
    verifier.verify(hash_obj, signature)

    ```


---

## 🔑 요점 정리 (시험 대비)

- 서명자(송신자)는 DSA 개인키로 `r, s` 서명 생성
- 수신자는 공개키로 `r, s` 검증
- 단순 해시 비교가 아닌 DSS 계산 과정을 통해 검증
- Python에서는 `pycryptodome`의 `DSS` 모듈이 이를 캡슐화함

In [5]:
# DSS 서명 내부의 r, s 값 직접 추출을 위해, hash + 서명 생성은 같은 방식으로 유지
from Crypto.Signature import DSS
from Crypto.PublicKey import DSA
from Crypto.Hash import SHA256
from Crypto.Random import random
import hashlib

# 키 생성
key = DSA.generate(2048)
public_key = key.publickey()

# 메시지 및 해시
message = b"This is an example of DSS signature."
hash_obj = SHA256.new(message)
hash_int = int.from_bytes(hash_obj.digest(), byteorder='big')

# 서명 수동 구현 (r, s 추출용)
# DSA 파라미터
p = key.p
q = key.q
g = key.g
x = key.x  # 개인키
y = key.y  # 공개키

# 랜덤 k 선택
k = random.StrongRandom().randint(1, q - 1)
r = pow(g, k, p) % q
k_inv = pow(k, -1, q)
s = (k_inv * (hash_int + x * r)) % q

(r, s)


(4964705246716120317936576436266479142571036899256270148230735566564,
 13544101167426954143072587103451853359394580228953721291748180836356)

## 🔐 서명 값 (DSA / ElGamal 기반)

```python
r = 800866696568881634641223408089620228137653158240314537017402473541
s = 11773709444316803657641576922151735245649239870198662392641058399230

```

이 두 값 `(r, s)`가 바로 전자서명의 핵심이며, 수신자는 이 값을 사용해 `v == r`인지 검증합니다.

---

## 📌 ElGamal 관련 설명

> ElGamal과 DSS 관계 요약
>
- *DSS(DSA)**는 ElGamal 서명 방식에서 **암호화 기능을 제거하고 서명 기능에 특화**시킨 형태입니다.
- 차이점은:
    - ElGamal은 암호화 + 서명 모두 지원
    - DSS(DSA)는 오직 서명만 지원 (암호화 없음)
- 내부 구조(지수연산, 모듈러 역원 사용 등)는 ElGamal을 바탕으로 하며,
    - 특히 r=(gkmod  p)mod  qr = (g^k \mod p) \mod qr=(gkmodp)modq 형식은 ElGamal 서명에서 유래

---

## 🧠 시험용 정리

| 알고리즘 | 암호화 | 서명 | 특징 |
| --- | --- | --- | --- |
| **ElGamal** | ✅ | ✅ | 지수 기반, 암호/서명 모두 |
| **DSS (DSA)** | ❌ | ✅ | ElGamal 서명 특화, 미국 표준 |

## 🧭 ElGamal 서명 방식 전체 흐름도

```
┌──────────────────────┐
│  서명자 (송신자)      │
└──────────────────────┘
        │
        ▼
1. 해시: H(M)
        │
        ▼
2. 무작위 k 선택 (k ⊥ (q-1))
        │
        ▼
3. r = g^k mod p
        │
        ▼
4. s = k⁻¹ · (H(M) - x·r) mod (p-1)
        │
        ▼
5. 메시지 + (r, s) 전송
        │
        ▼
┌──────────────────────┐
│  검증자 (수신자)      │
└──────────────────────┘
        │
        ▼
1. H(M) 계산
        │
        ▼
2. v1 = y^r · r^s mod p
3. v2 = g^H(M) mod p
        │
        ▼
4. v1 ≟ v2 → 같으면 검증 성공

```

---

## 📘 주요 변수 설명

| 변수               | 의미                                    |
|------------------|---------------------------------------|
| $p$              | 큰 소수                                  |
| $g$              | 생성기 (generator)                       |
| $x$              | 개인키                                   |
| $y = g^x \mod p$ | 공개키                                   |
| $k$              | 랜덤 정수 (서명마다 다르게)                      |
| $r$              | $g^k \mod p$                          |
| $s$              | $k^{-1}(H(M) - x \cdot r) \mod (p-1)$ |

---

## 🧪 검증자(수신자)의 검증 수식

- **v1 = $y^r \cdot r^s \mod p$**
- **v2 = $g^{H(M)} \mod p$**
- **비교: v1 == v2** → 서명 유효

---

## 🎯 DSS와 차이점 요약

| 항목 | ElGamal 서명                                     | DSS (DSA)                            |
| --- |------------------------------------------------|--------------------------------------|
| 서명 생성 수식 | $s = k^{-1}(H(M) - x·r)$                       | $s = k^{-1}(H(M) + x·r)$             |
| 소수 범위 | mod $p−1$                                      | mod $q$ (q는 p의 소인수)                  |
| 목적 | 암호 & 서명 모두                                     | 서명 전용                                |
| 검증 수식 | $y^r·r^s \mod p$ vs $g^{H(M)} \mod p$ | 복잡한 $u_1, u_2$ 계산 후 $v=r?$ |

# 📌 함정 보기 정리

> "전송자(서명자)와 수신자(검증자)가 공유한 비밀 정보를 이용하여 서명하여야 한다."
>
>
> → ❌ **틀린 보기입니다.**
>

---

## ✅ 이유: 전자서명은 비밀 공유를 **요구하지 않습니다**

전자서명 시스템(특히 **ElGamal, DSA, RSA 서명**)은 **공개키 기반 구조**이므로:

### 📌 핵심 개념

| 항목 | 설명 |
| --- | --- |
| 서명자(전송자) | 자신의 **개인키**로 서명 |
| 수신자(검증자) | 서명자의 **공개키**로 검증 |
| 비밀 공유 여부 | **공유된 비밀 없음**! 🔥 |

→ 즉, **서명자만 개인키를 알고** 수신자는 **공개키만 알면 됨**

---

## 🔒 반례: 비밀 공유는 대칭키 구조

공유된 비밀을 사용하는 경우는 보통:

- **대칭키 암호** (ex. AES)
- **메시지 인증 코드 (MAC)**
    - 예: HMAC, CMAC 등
    - ▶️ 여기서는 **공유 비밀키** 필요

전자서명은 이와 **정반대**임:

- 수신자는 **비밀 없이**도 검증 가능
- 이를 통해 → **발신자 인증 + 부인방지** 실현

---

## 🎯 시험 대비 정리

| OX 보기 문장 | 정답 여부 | 이유 |
| --- | --- | --- |
| 서명자는 개인키로 서명하고 수신자는 공개키로 검증한다. | ✅ | 전자서명의 기본 구조 |
| 전자서명은 송신자와 수신자가 공유한 비밀키로 서명한다. | ❌ | 공개키 기반 구조이므로 비밀 공유 불필요 |
| 메시지 인증 코드(MAC)는 공유 비밀키를 필요로 한다. | ✅ | 대칭키 방식 |

# 🔒 기밀성을 추가하려면?

$E_K[M ∥ E_{K{S_a}}[H(M)]]$

이 구조는 **전자서명 + 기밀성 보장**을 동시에 달성하기 위한 방식으로, **하이브리드 암호 시스템과 결합된 구조**입니다.

---

## 🔐 구조 해석

| 기호                  | 의미 |
|---------------------| --- |
| $M$                 | 원본 메시지 |
| $H(M)$              | 메시지의 해시 |
| $K_{Sa}$            | 서명자의 개인키 |
| $E_{K_{Sa}}[H(M)]$  | 전자서명 (개인키로 해시 암호화) |
| $∥$                 | 결합 (concatenation) |
| $K$                 | 대칭키 (세션키) |
| $E_K[ \cdot ]$ | 전체 메시지를 대칭키로 암호화 |

---

## 🔄 전체 흐름 (정리)

1. **전자서명 단계**
    - $H(M)$을 계산
    - $E_{K_{Sa}}[H(M)]$: 서명자의 개인키로 서명 생성
2. **메시지 + 서명 결합**
    - $M \| \text{전자서명}$M∥전자서명
3. **기밀성 추가**
    - 위 결과를 **대칭키 KKK**로 암호화
    - 결과:

        $E_K\left[M \,\|\, E_{K_{Sa}}[H(M)]\right]$


---

## ✅ 목적별 구성 요소

| 요소            | 목적 | 방식 |
|---------------| --- | --- |
| $E_{K_{Sa}}[H(M)]$ | **무결성 + 인증** (전자서명) | 비대칭 |
| $E_K[\cdot]$  | **기밀성 보호** | 대칭 |
| 전체 구조         | **하이브리드 암호 시스템 결합** | 전자서명 + 대칭 암호 |

---

## 📌 요약: 시험용 정리

- 이 구조는 **전자서명과 하이브리드 암호 시스템을 결합한 것**
- 서명 **완료 후** 전체를 **대칭키로 암호화**한 구조
- 기밀성 + 무결성 + 부인방지를 동시에 달성 가능

In [6]:
# 전자서명 + 대칭키 암호화 (하이브리드 구조)

from Crypto.PublicKey import DSA
from Crypto.Signature import DSS
from Crypto.Hash import SHA256
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import base64

# 1. 서명자 키 생성 (DSA)
dsa_key = DSA.generate(2048)
public_key = dsa_key.publickey()

# 2. 메시지와 해시
message = b"This is a confidential and signed message."
hash_obj = SHA256.new(message)

# 3. 전자서명 생성 (비대칭)
signer = DSS.new(dsa_key, 'fips-186-3')
signature = signer.sign(hash_obj)

# 4. 메시지 + 서명을 결합
combined = message + b'||' + signature  # 구분자 || 사용

# 5. 대칭키 생성 및 AES 암호화 (하이브리드 구조)
session_key = get_random_bytes(16)  # AES-128
cipher = AES.new(session_key, AES.MODE_EAX)
ciphertext, tag = cipher.encrypt_and_digest(combined)

# 암호화된 출력값
encrypted_package = {
    'ciphertext': base64.b64encode(ciphertext).decode(),
    'nonce': base64.b64encode(cipher.nonce).decode(),
    'tag': base64.b64encode(tag).decode(),
    'session_key': base64.b64encode(session_key).decode(),  # 실제 사용 시 비공개 / 공개키로 암호화 필요
}

encrypted_package


{'ciphertext': 'CrdCxVnR7/aV0QBBt1aTp+/i7B9jZKBf1/S2xDD9h4UBKum7uWIrmaVjzYnOEO14H7TyOB48uWmXk4LADiNH8Cl6WM9/qh8hubpqfxNroHQi0G9I6wMl+UxEhuzW+lpq962/NA==',
 'nonce': 'bKUUY/KS96J6/ePDPphdXg==',
 'tag': '+WaOMhfmoqpJG3njnLcu4A==',
 'session_key': 'NJJMQnbSYjNt47k7/o7E7w=='}

## 🔒 구성 설명

- `ciphertext`:

    → **[ 메시지 || 전자서명 ]** 전체를 AES로 암호화한 결과

- `nonce`, `tag`:

    → AES EAX 모드에서 사용하는 **무결성 확인용 요소**

- `session_key`:

    → **세션키 (대칭키)** — 실제 전송 시에는 **수신자의 공개키로 암호화**해서 보내야 안전함


---

## 📌 구조 요약

| 단계 | 암호 방식 | 사용 키 | 설명 |
| --- | --- | --- | --- |
| ① 메시지 해시 후 개인키로 서명 | 비대칭 (전자서명) | 개인키 | 무결성 + 인증 |
| ② 메시지와 서명을 묶음 | - | - | 서명 결합 |
| ③ 전체 묶음을 대칭키로 암호화 | 대칭키 (AES) | 세션키 | 기밀성 |
| ④ 세션키를 수신자 공개키로 암호화 (미구현) | 비대칭 | 수신자 공개키 | 세션키 보호 (하이브리드 구조 완성) |

In [7]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

# 6. 수신자 공개키 생성 (RSA)
rsa_receiver_key = RSA.generate(2048)
rsa_public_key = rsa_receiver_key.publickey()

# 7. 송신자가 세션 키를 수신자의 공개키로 암호화
rsa_cipher_enc = PKCS1_OAEP.new(rsa_public_key)
encrypted_session_key = rsa_cipher_enc.encrypt(session_key)

# 8. 수신자가 세션 키를 개인키로 복호화
rsa_cipher_dec = PKCS1_OAEP.new(rsa_receiver_key)
decrypted_session_key = rsa_cipher_dec.decrypt(encrypted_session_key)

# 9. 복호화 확인
correct = decrypted_session_key == session_key
correct


True

## ✅ 설명

> 사용자의 질문: "위에가 이미 수신자 공개키로 세션키 암호화한 거 아니에요?"
>
>
> → **이제 방금 한 게 그 과정입니다.**
>

---

## 🔐 지금까지 구성된 구조:

### 🔒 송신자

1. 메시지에 전자서명 생성 (개인키)
2. 메시지 + 서명 묶어서 AES로 암호화 (세션키)
3. 세션키를 **수신자의 공개키(RSA)** 로 암호화

### 🔓 수신자

1. **개인키(RSA)** 로 세션키 복호화
2. 세션키로 메시지 + 서명 복호화
3. **서명 검증** (송신자의 공개키로)

---

## ✅ 복호화 결과

```python
correct = decrypted_session_key == session_key  → True ✅

```

즉, 수신자가 **자신의 개인키로 세션키 복호화에 성공**했음을 의미합니다.

---

## 📌 전체 암호 흐름 요약도 (하이브리드 + 서명 결합)

```
[전자서명: 개인키로 H(M)]     🔐 인증/무결성
           │
           ▼
[M || 서명]                      메시지와 서명 결합
           │
           ▼
AES(K) ⟶ 암호화                 🔒 기밀성
           │
           ▼
세션키 K ⟶ RSA(PU_B)로 암호화   🔐 키 전송 보호

```

---

## $Q$ 메시지 인증 코드와 전자서명에 대한 설명으로 옳은 것은?

1) 전자서명은 대칭키가 사전에 교환되어야 사용할 수 있다.
2) 메시지 인증 코드와 전자서명 모두 무결성과 부인방지 기능을 제공한다.
3) 전자서명은 서명 생성자를 인증하는 기능이 있다.
4) 메시지 인증 코드값을 검증하는데 공개키가 필요하다.
5) 전자서명은 서명 후 해시 방식이다.

---

### ✅ 정답: **3) 전자서명은 서명 생성자를 인증하는 기능이 있다.**

---

## 📌 보기별 해설

### ❌ 1) 전자서명은 대칭키가 사전에 교환되어야 사용할 수 있다.

- **틀림**.

    전자서명은 **공개키 기반**입니다.

    → **대칭키 필요 없음**, 사전 키 공유도 필요 없음.


---

### ❌ 2) 메시지 인증 코드와 전자서명 모두 무결성과 부인방지 기능을 제공한다.

- **틀림**.
    - **무결성**: 둘 다 제공함
    - **부인방지**: **전자서명만 제공**
        - MAC은 **공유 비밀키 기반** → 누가 생성했는지 특정 불가 ❌

---

### ✅ 3) 전자서명은 서명 생성자를 인증하는 기능이 있다.

- **맞음**.

    전자서명은 송신자의 **개인키**로 생성되므로,

    수신자는 **공개키**로 복호화하여 서명자를 식별할 수 있음

    → **발신자 인증 가능**


---

### ❌ 4) 메시지 인증 코드값을 검증하는데 공개키가 필요하다.

- **틀림**.

    MAC은 **공개키 X**, **공유 대칭키** 사용

    → 비공개 키 기반 (공개키 기반 아님)


---

### ❌ 5) 전자서명은 서명 후 해시 방식이다.

- **틀림**.

    → **정확한 순서: 해시 후 서명**

    $M \rightarrow H(M) \rightarrow \text{전자서명}$

    서명 전 해시를 통해 데이터 양 줄이기 + 고정 길이화


---

## 🎯 헷갈릴 때 정리법

| 항목 | MAC (HMAC 등) | 전자서명 |
| --- | --- | --- |
| 키 | **대칭 (공유)** | **비대칭 (개인키/공개키)** |
| 무결성 | ✅ | ✅ |
| 발신자 인증 | ❌ | ✅ |
| 부인방지 | ❌ | ✅ |
| 검증 방식 | 같은 키 | **공개키** |

---

다음에 비슷한 문제가 나오면, **"MAC은 공유 비밀키 / 전자서명은 공개키 사용"**, **"전자서명은 인증 + 부인방지까지!"**

## ✅ MAC (Message Authentication Code)

- **키의 종류**:

    → **공유 비밀키**입니다.

    → **대칭키 방식의 키**


---

## 📌 정리

| 표현 | 설명 | 시험에서 표현될 수 있음 |
| --- | --- | --- |
| **대칭키** | 송신자와 수신자가 같은 키를 사용 | ✅ 맞는 표현 |
| **공유 비밀키** | 둘만 알고 있는 비밀 키 | ✅ 맞는 표현 |
| **공개키** | ❌ 아님 | 틀린 보기 유도용 표현 |

---

## 🎯 기억 포인트

- ✅ **MAC = 대칭키 = 공유 비밀키**
- ❌ **MAC은 공개키 암호 아님**
- 🔒 MAC은 키가 없으면 검증 불가 → **서로 비밀을 공유해야 함**
- ✍️ 전자서명은 **공개키 사용**

---

### 예시 문제 보기

> "메시지 인증 코드를 검증하려면 공개키가 필요하다." → ❌
>
>
> "메시지 인증 코드는 공유 비밀키 기반으로 동작한다." → ✅
>